# Custom Memory

In [2]:
from pyhere import here
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath('../..')) # adding the absolute path

from openai import OpenAI
client = OpenAI()


## Text Only Memory

#### Fn: Helper `_get_messages_user_assistant()` 

In [42]:
from typing import List
import pandas as pd

mem_df = pd.DataFrame({"user": ["Hi", "Nothing"], "assistant": ["How can I help you?", "Bye"]})

def _get_messages_user_assistant(mem_df: pd.DataFrame, 
                                 user_key = "user", 
                                 assistant_key = "assistant") -> list[dict]:
    """Convert user-assistant DataFrame into a list of message dictionaries."""
    messages = []
    for i in range(len(mem_df)):
        if mem_df[user_key][i]:  # Check if 'user' cell is not empty
            messages.append({"role": user_key, "content": mem_df[user_key][i]})
        if mem_df[assistant_key][i]:  # Check if 'assistant' cell is not empty
            messages.append({"role": assistant_key, "content": mem_df[assistant_key][i]})
    return messages

# Example usage
mem_df = pd.DataFrame({"user": ["Hi", "Nothing"], "assistant": ["How can I help you?", "Bye"]})
messages = _get_messages_user_assistant(mem_df)
print(messages)

[{'role': 'user', 'content': 'Hi'}, {'role': 'assistant', 'content': 'How can I help you?'}, {'role': 'user', 'content': 'Nothing'}, {'role': 'assistant', 'content': 'Bye'}]


In [56]:
mem_df = pd.DataFrame({"user": ["Hi", "There"], "assistant": ["How can I...", "Hello"]})
mem_df.loc[1]

mem_df.loc[1, "assistant"] = "Ha"
# _get_messages_user_assistant(mem_df)

Ha


#### Main Fn

In [ ]:
from typing import List
import pandas as pd

def get_completions_text_mem(prompts: List[str]):
    
    msg: List[dict[str, str]]
    mem_df = pd.DataFrame({"user": [], "assistant": []})
    
    for i in range(len(prompts)):
        # Get user text 
        mem_df.loc[i] = [prompts[i], None] 
        # Convert to user-assistance messages 
        msg = _get_messages_user_assistant(mem_df)
        # Generate Response
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                *msg
            ]
        )
        # response_text = "Resp: " + user_texts[i] # For Debug
        response_text = completion.choices[0].message.content
        # Update memory DF
        mem_df.loc[i, "assistant"] = response_text
        
    return mem_df
    
get_completions_text_mem(["Hi, my name is Kittipos.", "What's my name?"])

,user,assistant
0,"Hi, my name is Kittipos.","Hello, Kittipos! How can I assist you today?"
1,What's my name?,Your name is Kittipos. How can I help you furt...
